In [3]:
import os
import math
import numpy as np
import mujoco as mj
from typing import Any, Dict, Optional, Tuple, Union
import gymnasium as gym
from rfc_utils.tools import get_expert
from rfc_utils.config import Config


In [4]:
cfg_id = '050100'

cfg = Config(cfg_id, False, create_dirs=False)

cfg


In [5]:
from agent_envs.humanoid_env import HumanoidTemplate


env = HumanoidTemplate(cfg)

env

this is the model_path mocap_v2.xml


Understaing Expert